In [ ]:
import os

model_id="psychAiD"

# set shot num
n_shot=1


# Clinical
test_path='data/psych/task4.jsonl'
val_path='data/psych/task4.jsonl'


# save path
output_dir='data/fewshot1_task4'

os.system('python -u ./src/generate_fewshot_psych.py \
    --n_shot={} \
    --model_id={} \
    --output_dir={}  \
    --val_path={} \
    --test_path={} > output.log 2>&1'.format(n_shot,model_id,output_dir,val_path,test_path))

In [ ]:
### QC
import json

nshot = 0
for task in ['1','2','3_1','4','5']:


    print('-'*25,task,'-'*25)
    for model in ['gpt-3.5-turbo','gpt-4o-mini','gpt-4','gemini-1.5-pro','glm4','hunyuan-lite','hunyuan-pro','minimax','spark-4ultra','baichuan4','deepseek','doubao-pro-32k','ernie-4-8k','moonshot-v1-32k','yi-large','qwen-max']: 
    
        if task == '5': nshot=0
        output_pth = './result/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        lines = []
        id = 1
        total = 0
        failed = []
        try:
            with open(output_pth, 'r', encoding="utf-8") as f:
                for line in f:
                    total += 1
                    data = json.loads(line)
                    answer_key = [key for key in data.keys() if 'answer_' in key][0]
                    if data[answer_key] == 'API call failed':
                        failed.append(id)
                    id += 1
            print(model,'\t',total,'\t',len(failed),'\t',failed)
        except:
            print(output_pth,'not exist')


In [ ]:
### reformat json
import json

nshot = 0

for task in ['1','2','3','4','5']:
    for model in ['gpt-3.5-turbo','gpt-4o-mini','gpt-4','gemini-1.5-pro','glm4','hunyuan-lite','hunyuan-pro','minimax','spark-4ultra','baichuan4','deepseek','doubao-pro-32k','ernie-4-8k','moonshot-v1-32k','yi-large','qwen-max']: 

        if task == '5': nshot = 0
        output_pth = './result/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        lines = []
        try:
            with open(output_pth, 'r', encoding="utf-8") as f:
                for line in f:
                    lines.append(json.loads(line))
        except:
            print(output_pth,'not exist')
            continue
        with open(output_pth.replace('result','result-refined').replace('task3_1','task3'), 'w', encoding="utf-8") as f:
            json.dump(lines, f, ensure_ascii=False, indent=4)
        output_pth = output_pth.replace('result','result-refined').replace('task3_1','task3')
        print(f'output to {output_pth}')

In [ ]:
import os


nshot = 1
for task in [1,3]:
    print('-'*50,'task{}'.format(task),'-'*50)

    for model in ['gpt-3.5-turbo','gpt-4o-mini','gpt-4','gemini-1.5-pro','glm4','hunyuan-lite','hunyuan-pro','minimax','spark-4ultra','baichuan4','deepseek','doubao-pro-32k','ernie-4-8k','moonshot-v1-32k','yi-large','qwen-max']: 
        
        print(model)
        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        dir_out = './result/PsychClinical/{}shot/{}_api/task{}'.format(nshot,model,task)

        if not os.path.exists(ans_path):
            print('Error:',ans_path,'not exist!!!')
            continue

        os.system('python -u ./src/calc_metrics.py \
            --ans_path={} \
            --dir_out={} \
            > output_calc.log 2>&1'.format(ans_path,dir_out))


In [ ]:
### calc metrics
import tqdm
import numpy as np 
import os
import json 

## 完整性
def compute_integrity(ans):
    cnt = 0
    if '病程标准' in ans: cnt += 1
    if '症状学标准' in ans: cnt += 1
    if '严重程度标准' in ans: cnt += 1
    if '排除标准' in ans: cnt += 1
    
    return cnt/4*100

## 准确性
import re
def find_time(text):
    # 使用正则表达式匹配时间描述
    time_patterns = [
        r'\d+年',
        r'\d+月',
        r'\d+天',
        # r'\d+余',
        r'\d+余日',
        r'\d+余月',
        r'\d+余年'
    ]

    times_found = []
    for pattern in time_patterns:
        times_found.extend(re.findall(pattern, text))

    # 去重并排序结果
    unique_times = sorted(set(times_found))
    return unique_times

def find_des(text):
    # 使用正则表达式匹配时间描述
    des_patterns = [
        r'慢性',
        r'急性',
        r'持续',
        r'连续',
        r'间断',
        r'亚急性',
        r'反复',
        r'波动',
    ]

    des_found = []
    for pattern in des_patterns:
        des_found.extend(re.findall(pattern, text))

    # 去重并排序结果
    unique_times = sorted(set(des_found))
    if '急性' in unique_times and '亚急性' in unique_times:
        unique_times = [a for a in unique_times if a != '急性']
    
    return unique_times

import re
def find_icd(text):
    # text = "患者的主要诊断是 F30.901，需要进行进一步的治疗。"
    icd_dict = {}
    with open('./data/psych/icd-10.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            icd_dict[data["ICD-10 Name"]] = data['ICD-10 Code']
    icd10_codes = re.findall(r'\bF\d+\.\d+\b', text)
    if len(icd10_codes) < 1:
        for name in icd_dict.keys():
            if name in text:
                return icd_dict[name]
        # print('failed to find icd in:',text)
        return ''
    return icd10_codes[0]  # 输出: ['F30.901']

def acc_task1(out,tgt):
    try:
        out_zs = out.split('主诉')[1].split('病程标准')[0].split('病例特点')[0].replace('：','').replace(':','').replace('\n','')
    except:
        out_zs = out.split('病程标准')[0]
        # print(out)
    tgt_zs = tgt.split('主诉：')[1].split('病例特点')[0].replace('\n','')
    
    hit = 0
    tgt_num = 0
    
    times_zs = find_time(tgt_zs)
    tgt_num += len(times_zs)
    for t in times_zs:
        if t in out_zs:
            hit += 1

    if '病程标准' in out:
        out_bc = out.split('病程标准')[1].split('症状学标准')[0].replace('：','').replace(':','').replace('\n','').replace('*','')
    else:
        out_bc = ''
    tgt_bc = tgt.split('病程标准：')[1].split('2.')[0].replace('\n','')
    key_bc =  find_time(tgt_bc) + find_des(tgt_bc)
    tgt_num += len(key_bc)
    for k in key_bc:
        if k in out_bc:
            hit += 1
    
    if '严重程度标准' in out:
        out_yz = out.split('严重程度标准')[1].split('排除标准')[0].replace('：','').replace(':','').replace('\n','').replace('*','')
    else:
        out_yz = ''
    tgt_yz = tgt.split('严重程度标准：')[1].split('4.')[0].replace('\n','')
    opt = ['严重','明显','轻微']
    key_yz =  [o for o in opt if o in tgt_yz]
    tgt_num += len(key_yz)
    for k in key_yz:
        if k in out_yz:
            hit += 1

    try:
        acc = hit / tgt_num
    except:
        acc = 0
        print(tgt)
    return acc*100

def acc_task2(out,tgt):
    out_icd = find_icd(out.split('精神科共病诊断：')[0])
    icd_dict = {}
    with open('./data/psych/icd-10.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            icd_dict[data["ICD-10 Name"]] = data['ICD-10 Code']
    tgt_icd = icd_dict[tgt.split('主要诊断：')[1].split('精神科共病诊断：')[0].replace(' ','').replace('\n','')]
    
    # print(out_icd,tgt_icd)
    acc = 0
    if out_icd[:3] == tgt_icd[:3]:
        acc = 0.5
    if out_icd == tgt_icd:
        acc = 1
    return acc*100


def refine_name(text):
    names = ['人格障碍',
            '双相情感障碍',
            '器质性精神障碍',
            '复发性抑郁障碍',
            '妄想性障碍',
            '广泛性焦虑障碍',
            '强迫性障碍',
            '心境障碍',
            '急性而短暂的精神病性障碍',
            '抑郁发作',
            '抑郁障碍',
            '焦虑障碍',
            '环性心境',
            '精神分裂症',
            '精神障碍',
            '脑器质性精神病',
            '躁狂发作',
            '躯体症状障碍',
            '酒精所致的精神行为障碍',
            '阿尔兹海默',
            '阿尔茨海默']
    for name in names:
        if name in text:
            return name
    if len(re.findall(r'\“(.*?)\”',text)) > 0:
        return re.findall(r'\“(.*?)\”',text)[0]
    return text

def acc_task3_1(out,tgt):
    
    out = out.replace('：',':').replace(',','，')
    try:
        out_list = re.findall(r'\[(.*?)\]', out)
        out_list = [out for out in out_list if out!= '疾病名称']
        out_main = out_list[0]
        out_dif1 = out_list[1]
        out_dif2 = out_list[2]

    except:
        try:
            out_main = out.split('主要诊断')[1].split('\n')[0].replace('[','').replace(']','').replace(':','')
            out_dif1 = out.split('<鉴别诊断1>')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
            out_dif2 = out.split('<鉴别诊断2>')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
        except:
            try:
                out_main_part = out.split('鉴别诊断:')[0]
                out_main = refine_name(out_main_part)
                if out_main == out_main_part:
                    out_main = ''
            except:
                out_main = ''
            try:
                out_dif1 = out.split('<鉴别诊断1>')[1].split('<鉴别诊断2>')[0].split('鉴别点:')[0].replace('，','')
            except:
                
                try:
                    out_dif1 = out.split('鉴别诊断:')[1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif1 = out_dif1[-2].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif1 = out_dif1[1].split(':')[0].split('，')[0].replace('，','')
                        elif '1' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('1')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif1 = out.split('鉴别诊断')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif1 = ''
            try:
                out_dif2 = out.split('<鉴别诊断2>')[1].split('鉴别点:')[0].replace('，','')
            except:
                # print(out)
                try:
                    out_dif2 = out.split('鉴别诊断:')[1].split('2')[-1].split('-')[-1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif2 = out_dif2[-1].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif2 = ''
                        elif '2' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('2')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif2 = out.split('鉴别诊断')[-1].replace('\n\n','\n').split('\n')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif2 = ''

    out_main = refine_name(out_main)
    out_dif1 = refine_name(out_dif1)
    out_dif2 = refine_name(out_dif2) 
    if fuzzy_match(out_main,tgt['main']):
        acc_main = 1
    else:
        acc_main = 0
    
    acc_diff = 0
    if fuzzy_match_list(out_dif1,tgt['diff']):
        acc_diff += 0.5
    if fuzzy_match_list(out_dif2,tgt['diff']):
        acc_diff += 0.5
    # print(out_main,'|',out_dif1,'|',out_dif2)
    return acc_main*100,acc_diff*100



from fuzzywuzzy import fuzz

def fuzzy_match(str1, str2):
    
    ratio = fuzz.ratio(str1, str2)
    if ratio > 50:
        return True
    else:
        return False
    
def fuzzy_match_list(str1, tgt_list):
    
    flag = False
    for str2 in tgt_list:
        if fuzzy_match(str1,str2):
            flag = True
            break
    return flag

def hit_task4(out,tgt):
    drugs = ['丙戊酸','伏硫西汀','利培酮','劳拉西泮','唑吡坦','喹硫平','地西泮','奋乃静','奥氮平','奥沙西泮','帕利哌酮','帕罗西汀','度洛西汀','拉莫三嗪','文拉法辛','曲唑酮','氟伏沙明','氟哌啶醇','氟西汀','氨磺必利','氯丙嗪','氯氮平','硝西泮','碳酸锂','米氮平','美金刚','舍曲林','艾司西酞普兰','阿戈美拉汀','阿立哌唑','鲁拉西酮','齐拉西酮']
    out = out.replace('，',',').replace(' ','').replace('\n\n','\n').replace('：',':')
    tgt = tgt.replace('，',',').replace(' ','')
    try:
        if '推荐药物' in out:
            out_chunks = '推荐药物'.join(out.split('推荐药物')[1:])
        else:
            out_chunks = out.split('\n')
            if len(out_chunks) == 1:
                out_chunks = out_chunks
            elif '分析' in out_chunks[0]:
                out_chunks = out_chunks[1:]
            # out_chunks = out.split('\n')[1:]
        
        if isinstance(out_chunks,str):
            out_chunks = [out_chunks]
        ans_idx = 0
        max_match = 0
        for idx,piece in enumerate(out_chunks):
            match = 0
            for drug in drugs:
                if drug in piece: match += 1
            if match>max_match:
                max_match = match
                ans_idx = idx

        # out_list = out_chunks[ans_idx].split(':')[-1].replace('。','') #.split(',')#.split('、')
        out_list = []
        for drug in drugs:
            if drug in out_chunks[ans_idx]:
                out_list.append(drug)
        out_list.sort(key=lambda x: out_chunks[ans_idx].find(x))
    except:
    #     out_list = out.split('分析')[0].split(',').split(' ')
    
    # if len(out_list[0]) > 6:
        out_list = []
        for drug in drugs:
            if drug in '\n'.join(out.split('推荐药物')[1:]):
                out_list.append(drug)
        out_list.sort(key=lambda x: ('\n'.join(out.split('推荐药物')[1:])).find(x))
    
    if len(out_list) ==0:
        out_list = []
        for drug in drugs:
            if drug in '\n'.join(out.split('\n')[1:]):
                out_list.append(drug)
        out_list.sort(key=lambda x: ('\n'.join(out.split('\n')[1:])).find(x))
        
    if len(out_list) ==0:  
        out_list = ['']
        # print('failed to match drug:',out)

    
    tgt_list = tgt.split(',')
    print(out_list,'|',tgt_list)
    
    o_in_t = [d for d in out_list if d in tgt_list]
    t_in_o = [d for d in tgt_list if d in out_list]
    acc = 0
    if out_list[0]==tgt_list[0]:
        acc = 1
    recall = len(t_in_o) / len(tgt_list)
    precision = len(o_in_t) / len(out_list)
    
    return acc*100,precision*100,recall*100


def task5_exam_acc(out,tgt):
    correct = 0
    ans = re.findall(r'[A-Z]\.', out)
    if len(ans)<len(tgt):
        ans = re.findall(r'[A-Z]', out)
        if len(ans)<len(tgt):
            # print(out)
            # print('-'*50)
            if len(ans) < 1:
                return 0
            else:
                for a,t in zip(ans,tgt[:len(ans)]):
                    if a==t:
                        correct += 1
                return correct/len(tgt)
    else:
        ans = [a.replace('.','') for a in ans]

    ans = ans[-len(tgt):]
    
    for a,t in zip(ans,tgt):
        if a==t:
            correct += 1
    return correct/len(tgt)



### calc

nshot = 0
metrics_list = []   
task3_ans = np.load('./data/psych/task3_ans.npy',allow_pickle=True).item()
  
for task in [1,2,3,4]:

    print('-'*100)
    print('task',task)
    for model in ['gpt-3.5-turbo','gpt-4o-mini','gpt-4','gemini-1.5-pro','glm4','hunyuan-lite','hunyuan-pro','minimax','spark-4ultra','baichuan4','deepseek','doubao-pro-32k','ernie-4-8k','moonshot-v1-32k','yi-large','qwen-max']: 
        
        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        if not os.path.exists(ans_path):
            print(ans_path,'not exist')
            continue
        # load data
        lst_tgt = []
        lst_out = []
        lst_idx = []
        lst_ipid = []
        option_qa = [[],[]]
        with open(ans_path, "r", encoding="utf-8") as f:
            answers = json.load(f)
        idx = 0
        for ans in answers:
            if 'question_type' not in ans.keys():
                ans['question_type'] = 'clinical'
            if '选择题' in ans['question_type']:
                option_qa[0].append(ans['answer'])
                option_qa[1].append(ans['model_answer'])
                idx += 1
            else:
                if ans['question_type'] == 'clinical':
                    if task == '5_exam':
                        lst_tgt.append(ans['answer'])
                    elif ans['conversations'][1]['from']=='gpt':
                        lst_tgt.append(ans['conversations'][1]['value'])
                    else:
                        lst_tgt.append(ans['conversations'][2]['value'])
                else:
                    lst_tgt.append(ans['answer'])
                # lst_out.append(ans['model_answer'])
                lst_out.append(ans['answer_0'])
                lst_ipid.append(ans['id'])
                lst_idx.append(idx)
                idx += 1

        # compute scores of each sample across entire dataset
        scores_all = {}
        # for tgt, out, idx in tqdm.tqdm(zip(lst_tgt, lst_out, lst_idx)):
        for tgt, out, idx, ipid in zip(lst_tgt, lst_out, lst_idx,lst_ipid):

            if out == 'API调用失败':
                continue
            # get sub-dict containing scores for each metric
            # task1
            if 'task1' in ans_path:
                metrics = ['integrity','Acc']
                scores = [compute_integrity(out),acc_task1(out,tgt)]
            
            # task2
            if 'task2' in ans_path:
                metrics = ['Acc']
                scores = acc_task2(out,tgt)
            
            # task3
            if 'task3' in ans_path:
                metrics = ['Acc_main','Acc_diff']
                tgt = task3_ans[ipid]
                # scores = acc_task3(out,tgt)
                try:
                    scores = acc_task3_1(out,tgt)
                except:
                    print(out)

            
            # task4
            if 'task4' in ans_path:
                metrics = ['Acc','precision','recall']
                # print('-'*50)
                # print(out)
                scores = hit_task4(out,tgt)


            # task5 exam
            if 'task5' in ans_path:
                metrics = ['Acc']
                # print('-'*50)
                # print(out)
                scores = task5_exam_acc(out,tgt)

            # append to master dict, dataset object
            scores_all[idx] = scores

        # print(metrics)
        print(model,np.mean(np.array(list(scores_all.values())),0))
        avgs = np.mean(np.array(list(scores_all.values())),0,keepdims=False)
        stds = np.std(np.array(list(scores_all.values())),0,keepdims=False)
        if len(metrics) > 1:
            for metric,avg,std in zip(metrics,avgs,stds):
                metrics_list.append({'Task':'task{}'.format(task),'Model': model, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
        else:
            metrics_list.append({'Task':'task{}'.format(task),'Model': model, 'Metric': metrics[0], 'Average': avgs, 'Standard Deviation': stds})

In [ ]:
import pandas as pd
import json
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def summarize_metrics_from_files(directory, file_pattern="metrics.json",metrics_list=None,nshot=None):
    
    all_metrics = pd.DataFrame(columns=['Task','Model','Metric', 'Average', 'Standard Deviation'])
    if metrics_list is None:
        metrics_list = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == file_pattern:
                
                file_path = os.path.join(root, file)
                
                
                with open(file_path, 'r') as f:
                    data = json.load(f)
                

                task = os.path.basename(root)
                
                model_name = root.split('\\')[-2].split('_')[0]
                
                
                for metric, scores in data.items():
                    avg = scores['avg']
                    std = scores['std']
                    metrics_list.append({'Task':task,'Model': model_name, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
    
    
    all_metrics = pd.concat([pd.DataFrame(metrics_list)], ignore_index=True)

    
    all_metrics.to_csv('summary_result_{}shot.csv'.format(nshot), index=False)
    return all_metrics


nshot = 1
directory_path = "./result/PsychClinical/{}shot".format(nshot)
summary_df = summarize_metrics_from_files(directory_path,metrics_list=metrics_list,nshot=nshot)
print(summary_df)


In [ ]:
import pandas as pd

# Load the uploaded files
table_df = pd.read_excel('./table_new_0shot.xlsx')
summary_result_df = pd.read_csv('./summary_result_1shot.csv')

# Display the first few rows of each dataframe to understand their structure
table_df.head(), summary_result_df.head()


In [ ]:
col_name_map = {
    'task1':{'BLEU':0,'ROUGE-L':1,'BERT':2,'integrity':3,'Acc':4},
    'task2':{'Acc':0},
    'task3':{'BLEU':0,'ROUGE-L':1,'BERT':2,'Acc_main':3,'Acc_diff':4},
    'task4':{'Acc':0,'recall':1,'precision':2},
    'task5':{'BLEU':0,'ROUGE-L':1,'BERT':2}
}



# Function to format the data as 'avg±std' and round to 2 decimal places
def format_data(avg, std):
    return f"{avg:.2f}±{std:.2f}"

# Update the matching logic to handle the differences in task names
def get_column_name(task, metric):
    # Handle special case for integrity metric
    name =  col_name_map[task][metric]
    if name == 0:
        name = task 
    else:
        name = f"{task}.{name}"
    # Handle other metrics by appending the metric name with a dot separator
    return name

# Iterate over each row in summary_result_df and update the corresponding cells in table_df
for index, row in summary_result_df.iterrows():
    model = row['Model']
    task = row['Task']  # Convert task1 to Task1 for matching
    metric = row['Metric']
    avg = row['Average']
    std = row['Standard Deviation']
    
    # Find the column name in table_df that matches the task and metric
    try:
        column_name = get_column_name(task, metric)
    except:
        continue
    if column_name in table_df.columns:
        # Find the row in table_df that matches the model
        model_row = table_df[table_df['model'] == model].index
        if not model_row.empty:
            # Update the cell with the formatted data
            table_df.at[model_row[0], column_name] = format_data(avg, std)


# 保存DataFrame为Excel文件
table_df.to_excel('./table_new_1shot.xlsx', index=False)

# Display the updated table_df
table_df
